Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In [9]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    secret_share = protocol.uniform(shape=(2,2))
    secret = protocol.reveal(secret_share)
    log.info(f"Player {communicator.rank} secret: \n{secret}")
    secret_share_inv = protocol.multiplicative_inverse(secret_share)
    secret_inv = protocol.reveal(secret_share_inv)
    log.info(f"Player {communicator.rank} secret inv: \n{secret_inv}")
    product_share = protocol.untruncated_multiply(secret_share, secret_share_inv)
    product = protocol.reveal(product_share)
    log.info(f"Player {communicator.rank} product: \n{product}")


In this example we are once again starting with randomly selected field elements and revealing them. We do not work with encoding or decoding the values in this case since the multiplicative inverse we are dealing with in this context is that associated speicfically with the field, and not relative to the encoded or decoded values we are using as a symatic structure for our fixed point arithemtic. 

After producing and revealing the shared secrets we calculate their multiplicative inverse on an elementwise basis in the field, and reveal it. Finally we perform the multiplication and reveal the product thus demonstrating that the correct multiplicative inverses have been calculated since the product is a matrix comprised entirely of 1s. 

In [10]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:62975 from tcp://127.0.0.1:62975.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:62975 from tcp://127.0.0.1:62976.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:62975 from tcp://127.0.0.1:62977.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 secret: 
[[9402096883685486128 11980057545456853250]
 [14625725789530557442 561049172490591931]]
INFO:root:Player 1 secret: 
[[9402096883685486128 11980057545456853250]
 [14625725789530557442 561049172490591931]]
INFO:root:Player 2 secret: 
[[9402096883685486128 11980057545456853250]
 [14625725789530557442 561049172490591931]]
INFO:root:Player 0 secret inv: 
[[1083449188915050189 9295841596114911609]
 [8017291310648530075 8804392299823538794]]
INFO:roo

[None, None, None]